<a href="https://colab.research.google.com/github/jsedoc/ConceptorDebias/blob/ACL-cleanup/WEAT/Copy_of_Copy_of_WEAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from itertools import combinations, filterfalse
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models.keyedvectors import KeyedVectors
import pandas as pd
import random
import sys
import pickle



# WEAT Algorithm
The Word Embeddings Association Test (WEAT), as proposed by Calikson et. al., is a statistical test analogous to the Implicit Association Test (IAT) which helps quantify human biases in textual data. WEAT uses the cosine similarity between word embeddings which is analogous to the reaction time when subjects are asked to pair two concepts they find similar in the IAT.  WEAT considers two sets of target words and two sets of attribute words of equal size. The null hypothesis is that there is no difference between the two sets of target words and the sets of attribute words in terms of their relative similarities measured as the cosine similarity between the embeddings. For example, consider the target sets as words representing *Career* and *Family* and let the two sets of attribute words be *Male* and *Female* in that order. The null hypothesis states that *Career* and *Family* are equally similar (mathematically, in terms of the mean cosine similarity between the word representations) to each of the words in the *Male* and *Female* word lists. 

REF: https://gist.github.com/SandyRogers/e5c2e938502a75dcae25216e4fae2da5



## Test Statistic

The WEAT test statistic measures the differential association of the two sets of target words with the attribute.

To ground this, we cast WEAT in our formulation where $\mathcal{X}$ and $\mathcal{Y}$ are two sets of target
words, (concretely, $\mathcal{X}$ might be*Career* words and $\mathcal{Y}$ *Family* words) and $\mathcal{A}$, $\mathcal{B}$ are two sets of attribute words ($\mathcal{A}$ might be ''female'' names and $\mathcal{B}$  ''male'' names) assumed to associate with the bias concept(s). WEAT is then~\footnote{We assume that there is no overlap between any of the sets $\mathcal{X}$, $\mathcal{Y}$, $\mathcal{A}$, and $\mathcal{B}$.} 
\begin{align*}
s(\mathcal{X}, &\mathcal{Y}, \mathcal{A}, \mathcal{B}) \\ &= \frac{1}{|\mathcal{X}|}\Bigg[\sum_{x \in \mathcal{X}}{\Big[\sum_{a\in \mathcal{A}}{s(x,a)} - \sum_{b\in \mathcal{B}}{s(x,b)}\Big]} \\ &\hbox{}  - \sum_{y \in \mathcal{Y}}{\Big[\sum_{a\in \mathcal{A}}{s(y,a)} - \sum_{b\in \mathcal{B}}{s(y,b)}\Big]}\Bigg],
\end{align*}
where $s(x,y) = \cos(\hbox{vec}(x), \hbox{vec}(y))$ and $\hbox{vec}(x) \in \mathbb{R}^k$ is the $k$-dimensional word embedding for word $x$. 
Note that for this definition of WEAT, the cardinality of the sets must be equal, so $|\mathcal{A}|=|\mathcal{B}|$ and $|\mathcal{X}|=|\mathcal{Y}|$. Our  conceptor formulation given below relaxes this assumption.

In [0]:
def swAB(W, A, B):
  """Calculates differential cosine-similarity between word vectors in W, A and W, B
     Arguments
              W, A, B : n x d matrix of word embeddings stored row wise
  """
  WA = cosine_similarity(W,A)
  WB = cosine_similarity(W,B)
  
  #Take mean along columns
  WAmean = np.mean(WA, axis = 1)
  WBmean = np.mean(WB, axis = 1)
  
  return (WAmean - WBmean)
  
def test_statistic(X, Y, A, B):
  """Calculates test-statistic between the pair of association words and target words
     Arguments
              X, Y, A, B : n x d matrix of word embeddings stored row wise
     Returns
              Test Statistic
  """
  return (sum(swAB(X, A, B)) - sum(swAB(Y, A, B)))

## Effect Size

The ''effect size'' is a normalized measure of how separated the two distributions are.

In [0]:
def weat_effect_size(X, Y, A, B, embd):
  """Computes the effect size for the given list of association and target word pairs
     Arguments
              X, Y : List of association words
              A, B : List of target words
              embd : Dictonary of word-to-embedding for all words
     Returns
              Effect Size
  """
  
  Xmat = np.array([embd[w.lower()] for w in X if w.lower() in embd])
  Ymat = np.array([embd[w.lower()] for w in Y if w.lower() in embd])
  Amat = np.array([embd[w.lower()] for w in A if w.lower() in embd])
  Bmat = np.array([embd[w.lower()] for w in B if w.lower() in embd])
  
  XuY = list(set(X).union(Y))
  XuYmat = []
  for w in XuY:
    if w.lower() in embd:
      XuYmat.append(embd[w.lower()])
  XuYmat = np.array(XuYmat)

  
  d = (np.mean(swAB(Xmat,Amat,Bmat)) - np.mean(swAB(Ymat,Amat,Bmat)))/np.std(swAB(XuYmat, Amat, Bmat))
  
  return d

## P-Value

The one-sided P value measures the likelihood that a random permutation of the attribute words would produce at least the observed test statistic

In [0]:
def random_permutation(iterable, r=None):
  """Returns a random permutation for any iterable object"""
  pool = tuple(iterable)
  r = len(pool) if r is None else r
  return tuple(random.sample(pool, r))

def weat_p_value(X, Y, A, B, embd, sample = 1000):
  """Computes the one-sided P value for the given list of association and target word pairs
     Arguments
              X, Y : List of association words
              A, B : List of target words
              embd : Dictonary of word-to-embedding for all words
              sample : Number of random permutations used.
     Returns
  """
  size_of_permutation = min(len(X), len(Y))
  X_Y = X + Y
  test_stats_over_permutation = []
  
  Xmat = np.array([embd[w.lower()] for w in X if w.lower() in embd])
  Ymat = np.array([embd[w.lower()] for w in Y if w.lower() in embd])
  Amat = np.array([embd[w.lower()] for w in A if w.lower() in embd])
  Bmat = np.array([embd[w.lower()] for w in B if w.lower() in embd])
  
  if not sample:
      permutations = combinations(X_Y, size_of_permutation)
  else:
      permutations = [random_permutation(X_Y, size_of_permutation) for s in range(sample)]
      
  for Xi in permutations:
    Yi = filterfalse(lambda w:w in Xi, X_Y)
    Ximat = np.array([embd[w.lower()] for w in Xi if w.lower() in embd])
    Yimat = np.array([embd[w.lower()] for w in Yi if w.lower() in embd])
    test_stats_over_permutation.append(test_statistic(Ximat, Yimat, Amat, Bmat))
    
  unperturbed = test_statistic(Xmat, Ymat, Amat, Bmat)
  
  is_over = np.array([o > unperturbed for o in test_stats_over_permutation])
  
  return is_over.sum() / is_over.size

## Test on sample input

In [0]:
"""List of association and target word pairs for the sample test (Instruments, Weapons) vs (Pleasant, Unpleasant)"""

# Instruments
X = ["bagpipe", "cello", "guitar", "lute", "trombone", "banjo", "clarinet", "harmonica", "mandolin", "trumpet", "bassoon", "drum", "harp", "oboe", "tuba", "bell", "fiddle", "harpsichord", "piano", "viola", "bongo",
"flute", "horn", "saxophone", "violin"] 
# Weapons
Y = ["arrow", "club", "gun", "missile", "spear", "axe", "dagger", "harpoon", "pistol", "sword", "blade", "dynamite", "hatchet", "rifle", "tank", "bomb", "firearm", "knife", "shotgun", "teargas", "cannon", "grenade",
    "mace", "slingshot", "whip"] 
# Pleasant
A = ["caress", "freedom", "health", "love", "peace", "cheer", "friend", "heaven", "loyal", "pleasure", "diamond", "gentle", "honest", "lucky", "rainbow", "diploma", "gift", "honor", "miracle", "sunrise", "family",
    "happy", "laughter", "paradise", "vacation"] 
# Unpleasant
B = ["abuse", "crash", "filth", "murder", "sickness", "accident", "death", "grief", "poison", "stink", "assault", "disaster", "hatred", "pollute", "tragedy", "divorce", "jail", "poverty", "ugly", "cancer", "kill", "rotten",
    "vomit", "agony", "prison"] 

"""Download the 'Glove' embeddings if not downloaded"""
!if [ ! -f /content/gensim_glove.840B.300d.txt.bin ]; then gdown https://drive.google.com/uc?id=1Ty2exMyi-XOufY-v81RJfiPvnintHuy2; fi

"""Load the embeddings to a gensim object"""
resourceFile = ''
glove = KeyedVectors.load_word2vec_format(resourceFile + 'gensim_glove.840B.300d.txt.bin', binary=True)
print('The glove embedding has been loaded!')

"""Compute the effect-size and P value"""
print('WEAT d = ', weat_effect_size(X, Y, A, B, glove))
print('WEAT p = ', weat_p_value(X, Y, A, B, glove, 1000))

The glove embedding has been loaded!
WEAT d =  1.5495627
WEAT p =  0.0


# Load all vectors and compute the conceptor

Clone our repository. The repository contains code for computing the conceptors. It also includes word lists needed representing different subspaces.

In [0]:
# our code for debiasing -- also includes word lists
!rm -r ConceptorDebias
!git clone https://github.com/jsedoc/ConceptorDebias
!cd ConceptorDebias; git checkout ACL-cleanup

sys.path.append('/content/ConceptorDebias')

from Conceptors.conceptor_fxns import *

## Compute the conceptor matrix

In [0]:
#Compute the conceptor matrix
def process_cn_matrix(subspace, alpha = 2):
  
  C = train_conceptor(subspace, alpha)
  
  #Calculate the negation of the conceptor matrix
  negC = NOT(C)
  
  return negC

def apply_conceptor(x, C):
  #Post-process the vocab matrix
  newX = (C @ x).T
  print(newX.shape)
  return newX

## Load embeddings of all words from the ref. wordlist from a specific embedding

In [0]:
#Arguments - embd: The word embeddings in the form of a dict || wikiWordsPath: List of words to be considered
def load_all_vectors(embd, wikiWordsPath):
  all_words_index = {}
  all_words_mat = []
  with open(wikiWordsPath, "r+") as f_in:
    ind = 0
    for line in f_in:
      word = line.split(' ')[0]
      if word in embd:
        all_words_index[word] = ind
        all_words_mat.append(embd[word])
        ind = ind+1
        
  return all_words_index, all_words_mat

def load_subspace_vectors(embd, subspace_words):
  subspace_embd_mat = []
  ind = 0
  for word in subspace_words:
    if word in embd:
      subspace_embd_mat.append(embd[word])
      ind = ind+1
      
  return subspace_embd_mat

## Load all word lists for the ref. subspace

In [0]:
# General word list
!wget https://raw.githubusercontent.com/IlyaSemenov/wikipedia-word-frequency/master/results/enwiki-20150602-words-frequency.txt
!git clone https://github.com/PrincetonML/SIF
    
# Gender word lists
!git clone https://github.com/uclanlp/gn_glove
!git clone https://github.com/uclanlp/corefBias
!wget https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/female.txt
!wget https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/male.txt

from lists.load_word_lists import *

In [0]:
gender_list_pronouns = WEATLists.W_7_Male_terms + WEATLists.W_7_Female_terms + WEATLists.W_8_Male_terms + WEATLists.W_8_Female_terms
gender_list_pronouns = list(set(gender_list_pronouns))

gender_list_extended = male_vino_extra + female_vino_extra + male_gnGlove + female_gnGlove
gender_list_extended = list(set(gender_list_extended))

gender_list_propernouns = male_cmu + female_cmu
gender_list_propernouns = list(set(gender_list_propernouns))

gender_list_all = gender_list_pronouns + gender_list_extended + gender_list_propernouns
gender_list_all = list(set(gender_list_all))

race_list = WEATLists.W_3_Unused_full_list_European_American_names + WEATLists.W_3_European_American_names + WEATLists.W_3_Unused_full_list_African_American_names + WEATLists.W_3_African_American_names + WEATLists.W_4_Unused_full_list_European_American_names + WEATLists.W_4_European_American_names + WEATLists.W_4_Unused_full_list_African_American_names + WEATLists.W_4_African_American_names + WEATLists.W_5_Unused_full_list_European_American_names + WEATLists.W_5_European_American_names + WEATLists.W_5_Unused_full_list_African_American_names + WEATLists.W_5_African_American_names 
race_list = list(set(race_list))

## Load different embeddings

Glove

In [0]:
#Load word embeddings
#download gensim formatted Full Glove embeddings
!if [ ! -f /content/gensim_glove.840B.300d.txt.bin ]; then gdown https://drive.google.com/uc?id=1Ty2exMyi-XOufY-v81RJfiPvnintHuy2; fi

from gensim.models.keyedvectors import KeyedVectors

resourceFile = ''

if 'glove' not in dir():
  glove = KeyedVectors.load_word2vec_format(resourceFile + 'gensim_glove.840B.300d.txt.bin', binary=True)
  print('The glove embedding has been loaded!')

In [0]:
"""Sample output of the glove embeddings"""
X = WEATLists.W_5_Unused_full_list_European_American_names
print(X)
a = [glove[w] for w in X if w.lower() in glove]
print(np.array(a).shape)
glove['Brad']
#glove['brad']

['Brad', 'Brendan', 'Geoffrey', 'Greg', 'Brett', 'Jay', 'Matthew', 'Neil', 'Todd', 'Allison', 'Anne', 'Carrie', 'Emily', 'Jill', 'Laurie', 'Kristen', 'Meredith', 'Sarah']
(18, 300)


array([-2.5609e-01, -1.5396e-01, -5.0346e-01,  2.5138e-01,  3.3952e-01,
        2.6105e-01,  3.0257e-01, -1.1273e-01,  4.6391e-02,  2.1197e+00,
        1.9856e-01, -3.1840e-02, -4.4469e-01, -8.6130e-03, -3.6665e-01,
       -7.8715e-02, -1.8301e-01, -3.5935e-01, -2.9033e-01,  3.2755e-01,
       -1.1633e-01, -5.3637e-02, -3.1589e-01, -9.0069e-02,  5.4889e-02,
        1.0793e-01, -7.6549e-02,  3.2448e-01, -3.7830e-01, -7.1353e-03,
       -9.2837e-02, -3.4702e-01, -4.4372e-02, -3.6582e-02,  6.6447e-02,
        4.3170e-01,  1.4958e-01,  4.1407e-01, -7.3515e-04, -5.4028e-01,
        3.4552e-01, -5.6781e-02, -1.9483e-01,  3.7953e-01,  4.6593e-02,
        4.1634e-02, -5.3288e-01,  1.4703e-01,  2.6140e-02, -1.6343e-03,
        3.7304e-01, -1.7348e-01,  2.5698e-02, -2.5498e-01, -1.9006e-01,
       -5.8404e-01, -6.0172e-03,  4.6960e-02,  1.4949e-02,  4.7884e-01,
        1.8709e-01,  1.1012e-01, -2.9987e-01,  3.7356e-02, -4.1855e-02,
        1.2758e-02,  1.5549e-01, -7.8430e-02,  3.5795e-01, -2.11

Word2vec

In [0]:
#load gensim formatted Full Word2vec embeddings
!if test -e /content/GoogleNews-vectors-negative300.bin.gz || test -e /content/GoogleNews-vectors-negative300.bin; then echo 'file already downloaded'; else echo 'starting download'; gdown https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM; fi
!if [ ! -f /content/GoogleNews-vectors-negative300.bin ]; then gunzip GoogleNews-vectors-negative300.bin.gz; fi
  
import gensim

from gensim.models.keyedvectors import KeyedVectors

resourceFile = ''

if 'word2vec' not in dir():
  word2vec = KeyedVectors.load_word2vec_format(resourceFile + 'GoogleNews-vectors-negative300.bin', binary=True)
  print('The word2vec embedding has been loaded!')

file already downloaded
The word2vec embedding has been loaded!


Fasttext

In [0]:
!if [ ! -f /content/fasttext.bin ]; then gdown https://drive.google.com/uc?id=1Zl6a75Ybf8do9uupmrJWKQMnvqqme4fh; fi

import gensim
from gensim.models.keyedvectors import KeyedVectors  

resourceFile = ''
if 'fasttext' not in dir():
  fasttext = KeyedVectors.load_word2vec_format(resourceFile + 'fasttext.bin', binary=True)
  print('The fasttext embedding has been loaded!')

The fasttext embedding has been loaded!


Elmo

In [0]:
!if [ ! -f /content/elmo_embeddings_emma_brown.pkl ]; then gdown https://drive.google.com/uc?id=17TK2h3cz7amgm2mCY4QCYy1yh23ZFWDU; fi

data = pickle.load(open("elmo_embeddings_emma_brown.pkl", "rb"))

def pick_embeddings(corpus, sent_embs):
    X = []
    labels = {}
    sents = []
    ind = 0
    for i, s in enumerate(corpus):
        for j, w in enumerate(s):
            X.append(sent_embs[i][j])
            if w.lower() in labels:
              labels[w.lower()].append(ind)
            else:
              labels[w.lower()] = [ind]
            sents.append(s)
            ind = ind + 1
    return (X, labels, sents)
  
def get_word_list(path):
    word_list = []
    with open(path, "r+") as f_in:
      for line in f_in:
        word = line.split(' ')[0]
        word_list.append(word.lower())

    return word_list

def load_subspace_vectors_contextual(all_mat, all_index, subspace_list):
    subspace_mat = []
    for w in subspace_list:
      if w.lower() in all_index:
        for i in all_index[w.lower()]:
          #print(type(i))
          subspace_mat.append(all_mat[i])
    #subspace_mat = [all_mat[i,:] for i in all_index[w.lower()] for w in subspace_list if w.lower() in all_index]
    print("Subspace: ", np.array(subspace_mat).shape)
    return subspace_mat

In [0]:
import nltk
from nltk.corpus import brown

#nltk.download('brown')

brown_corpus = brown.sents()
elmo = data['brown_embs']


In [0]:
print(np.array(elmo_brown_mat).shape)
print(len(list(elmo_brown_index.keys())))
print(len(wiki_words))

Bert

# WEAT on Conceptored embeddings

## Initialize variables

In [0]:
resourceFile = ''
wikiWordsPath = resourceFile + 'SIF/auxiliary_data/enwiki_vocab_min200.txt' # https://github.com/PrincetonML/SIF/blob/master/auxiliary_data/enwiki_vocab_min200.txt

"""Set the embedding to be used"""
embd = 'glove'

"""Set the subspace to be tested on"""
subspace = 'gender_list_all' 

"""Load association and target word pairs"""
X = WEATLists.W_8_Science
Y = WEATLists.W_8_Arts
A = WEATLists.W_8_Male_terms
B = WEATLists.W_8_Female_terms

## Load the vectors as a matrix

In [0]:
curr_embd = eval(embd)
  
"""Load all embeddings in a matrix of all words in the wordlist"""
if embd == 'elmo':
  all_words_mat, all_words_index, _ = pick_embeddings(brown_corpus, curr_embd)
else:
  all_words_index, all_words_mat = load_all_vectors(curr_embd, wikiWordsPath)
  


## Compute the conceptor

In [0]:
if subspace != 'without_conceptor':
  subspace_words_list = eval(subspace)
  if subspace == 'gender_list_and':
    if embd == 'elmo':
      subspace_words_mat1 = load_subspace_vectors_contextual(all_words_mat, all_words_index, gender_list_pronouns)
      cn1 = process_cn_matrix(np.array(subspace_words_mat1).T, alpha = 8)

      subspace_words_mat2 = load_subspace_vectors_contextual(all_words_mat, all_words_index, gender_list_extended)
      cn2 = process_cn_matrix(np.array(subspace_words_mat2).T, alpha = 3)

      subspace_words_mat3 = load_subspace_vectors_contextual(all_words_mat, all_words_index, gender_list_propernouns)
      cn3 = process_cn_matrix(np.array(subspace_words_mat3).T, alpha = 10)

      cn = AND(cn1, AND(cn2, cn3))
    else:
      subspace_words_mat1 = load_subspace_vectors(curr_embd, gender_list_pronouns)
      cn1 = process_cn_matrix(np.array(subspace_words_mat1).T)

      subspace_words_mat2 = load_subspace_vectors(curr_embd, gender_list_extended)
      cn2 = process_cn_matrix(np.array(subspace_words_mat2).T)

      subspace_words_mat3 = load_subspace_vectors(curr_embd, gender_list_propernouns)
      cn3 = process_cn_matrix(np.array(subspace_words_mat3).T)

      cn = AND(cn1, AND(cn2, cn3))
  else: 
    if embd == 'elmo':
      subspace_words_mat = load_subspace_vectors_contextual(all_words_mat, all_words_index, subspace_words_list)
      cn = process_cn_matrix(np.array(subspace_words_mat).T, alpha = 6)
    else:
      subspace_words_mat = load_subspace_vectors(curr_embd, subspace_words_list)
      cn = process_cn_matrix(np.array(subspace_words_mat).T)

## Compute conceptored embeddings

In [0]:
all_words_cn = apply_conceptor(np.array(all_words_mat).T, np.array(cn))

#Store all conceptored words in a dictonary
all_words = {}
for word, index in all_words_index.items():
  #print(word, index)
  if embd == 'elmo':
    all_words[word] = np.mean([all_words_cn[i,:] for i in index], axis = 0)
  else:
    all_words[word] = all_words_cn[index,:]

## Calculate WEAT scores

In [0]:
d = weat_effect_size(X, Y, A, B, all_words)
p = weat_p_value(X, Y, A, B, all_words, 1000)

print('WEAT d = ', d)
print('WEAT p = ', p)

# WEAT on BERT embeddings

In [0]:
import os

#Load all bert embeddings in a dictonary
all = {}
for filename in os.listdir('/home/saketk/bert'):
  all[filename] = pickle.load(open(filename, "rb"))

#Load all embeddings in a matrix and a dictonary of words to row numbers
res = all['big_bert_gender_list_extended.pkl']['type_embedings']
w = []
for name in all:
  res = np.concatenate((res, all[name]['type_embedings']))
  w += all[name]['words']
print(res.shape)
print(len(set(w)))
w = [aaa.lower() for aaa in w]

#Load all BERT conceptors
cn_pronouns = all['big_bert_gender_list_pronouns.pkl']['GnegC']
cn_propernouns = all['big_bert_gender_list_propernouns.pkl']['GnegC']
cn_extended = all['big_bert_gender_list_extended.pkl']['GnegC']
cn_all = all['big_bert_gender_list_all.pkl']['GnegC']
cn_race = all['big_bert_race_list.pkl']['GnegC']

print(np.array(cn_all).shape)

Test Debiasing on BERT Embeddings using WEAT

In [0]:
import numpy as np
from ConceptorDebias.conceptor_fxns import AND
import pandas as pd

resourceFile = ''
wikiWordsPath = resourceFile + 'SIF/auxiliary_data/enwiki_vocab_min200.txt' # https://github.com/PrincetonML/SIF/blob/master/auxiliary_data/enwiki_vocab_min200.txt

all_embd = ['bert']
all_subspace = ['without_conceptor','gender_list_pronouns', 'gender_list_extended','gender_list_propernouns', 'gender_list_all', 'gender_list_and']
# all_subspace = ['without_conceptor', 'race_list']

all_words_mat = res
all_words_index = {}
print(len(set(w)))
for i,a in enumerate(w):
  all_words_index[a] = i
# all_words_index = {wr:i for i,wr in enumerate(w)}
print("All_words_mat: ", all_words_mat.shape)
print("Index: ", len(all_words_index.keys()))
career = WEATLists.W_8_Science
family = WEATLists.W_8_Arts
male = WEATLists.W_8_Male_terms
female = WEATLists.W_8_Female_terms

# white = WEATLists.W_5_Unused_full_list_European_American_names
# black = WEATLists.W_5_Unused_full_list_African_American_names
# pleasant = WEATLists.W_5_Pleasant
# unpleasant = WEATLists.W_5_Unpleasant
embd = 'bert'
results = []
for subspace in all_subspace:
    
  if subspace != 'without_conceptor' and subspace != 'gender_list_and':
    subspace_words_list = eval(subspace)

  if subspace != 'without_conceptor':
    #CN all word embeddings using the respective subspace
    if subspace == 'gender_list_and':
      cn = AND(cn_pronouns, AND(cn_extended, cn_propernouns))
      all_words_cn = apply_conceptor(np.array(all_words_mat).T, np.array(cn))
      print("All mat CN: ", np.array(all_words_cn).shape)
    else:
      #Load all embeddings of the subspace as a matrix
      if subspace == 'gender_list_pronouns':
        cn = cn_pronouns
      elif subspace == 'gender_list_propernouns':
        cn = cn_propernouns
      elif subspace == 'gender_list_extended':
        cn = cn_extended
      elif subspace == 'gender_list_all':
        cn = cn_all
        
      print("CN shape: ", np.array(cn).shape)
      all_words_cn = apply_conceptor(np.array(all_words_mat).T, np.array(cn))
      print("Subspace mat: ", np.array(subspace_words_mat).shape)
      
  else:
    all_words_cn = all_words_mat
  all_words_cn = np.array(all_words_cn)
  print("All CN: ", all_words_cn.shape)
  #Store all conceptored words in a dictonary
  all_words = {}
  for word, index in all_words_index.items():
    all_words[word] = all_words_cn[index,:]
  print("D: ", len(all_words.keys()))
  if subspace == 'without_conceptor':
    #WITHOUT CONCEPTOR
    d_cn = weat_effect_size(career, family, male, female, all_words)
    p_cn = weat_p_value(career, family, male, female, all_words, 1000)
    print("Without conceptor")
    print('WEAT d = ', d_cn)
    print('WEAT p = ', p_cn)
  else:
    #WITH CONCEPTOR
    d_cn = weat_effect_size(career, family, male, female, all_words)
    p_cn = weat_p_value(career, family, male, female, all_words, 1000)
    print("With conceptor: ", embd, subspace)
    print('WEAT d = ', d_cn)
    print('WEAT p = ', p_cn)

  row = [embd, subspace, d_cn, p_cn]
  results.append(row)

  
  
pd.DataFrame(np.array(results), columns = ['Embedding', 'Subspace', 'Effect Size', 'p-value'])

# Hard Debiasing

## Mu et. al. Hard Debiasing

In [0]:
from sklearn.decomposition import PCA

#Project off first principal component
def hard_debias(all_words, subspace):
  all_words = np.array(all_words)
  subspace = np.array(subspace)
  print(all_words.shape, " ", subspace.shape)
  pca = PCA(n_components = 1)
  pca.fit(subspace)
  pc1 = np.array(pca.components_)
  
  temp = (pc1.T @ (pc1 @ all_words.T)).T
  ret = all_words - temp
  
  return ret


Test Hard Debiasing using WEAT

In [0]:
import numpy as np
import pandas as pd


resourceFile = ''
wikiWordsPath = resourceFile + 'SIF/auxiliary_data/enwiki_vocab_min200.txt' # https://github.com/PrincetonML/SIF/blob/master/auxiliary_data/enwiki_vocab_min200.txt

all_embd = ['glove', 'word2vec', 'fasttext','elmo']
all_subspace = ['without_debiasing', 'gender_list_extended','gender_list_propernouns','gender_list_pronouns', 'gender_list_all']
# all_subspace = ['without_debiasing', 'race_list']

# science = WEATLists.W_8_Science
# arts = WEATLists.W_8_Arts
# male = WEATLists.W_8_Male_terms
# female = WEATLists.W_8_Female_terms

white = WEATLists.W_5_Unused_full_list_European_American_names
black = WEATLists.W_5_Unused_full_list_African_American_names
pleasant = WEATLists.W_5_Pleasant
unpleasant = WEATLists.W_5_Unpleasant

#print(career, family, male, female)
results = []

for embd in all_embd:
  #Initialize the embeddings to be used
  curr_embd = eval(embd)
  
  #Load all embeddings in a matrix of all words in the wordlist
  if embd == 'elmo' or embd == 'bert':
    all_words_mat, all_words_index, _ = pick_embeddings(brown_corpus, curr_embd)
    print("All mat: ", np.array(all_words_mat).shape)
    print("Number of words: ", len(list(all_words_index.keys())))
  else:
    all_words_index, all_words_mat = load_all_vectors(curr_embd, wikiWordsPath)
  
  for subspace in all_subspace:
    
    if subspace != 'without_debiasing' and subspace != 'gender_list_and':
      subspace_words_list = eval(subspace)

    
    if subspace != 'without_debiasing':
      #Load all embeddings of the subspace as a matrix
      if embd == 'elmo' or embd == 'bert':
        subspace_words_mat = load_subspace_vectors_contextual(all_words_mat, all_words_index, subspace_words_list)
        all_words_cn = hard_debias(all_words_mat, subspace_words_mat)
        print("Subspace mat: ", np.array(subspace_words_mat).shape)
      else:
        subspace_words_mat = load_subspace_vectors(curr_embd, subspace_words_list)
        all_words_cn = hard_debias(all_words_mat, subspace_words_mat)
    else:
      all_words_cn = all_words_mat
    all_words_cn = np.array(all_words_cn)
    print("All CN: ", all_words_cn.shape)
    #Store all conceptored words in a dictonary
    all_words = {}
    for word, index in all_words_index.items():
      #print(word, index)
      if embd == 'elmo' or embd == 'bert':
        all_words[word] = np.mean([all_words_cn[i,:] for i in index], axis = 0)
      else:
        all_words[word] = all_words_cn[index,:]
    
    if subspace == 'without_debiasing':
      #WITHOUT CONCEPTOR
      d_cn = weat_effect_size(white, black, pleasant, unpleasant, all_words)
      p_cn = weat_p_value(white, black, pleasant, unpleasant, all_words, 1000)
      print("Without debiasing")
      print('WEAT d = ', d_cn)
      print('WEAT p = ', p_cn)
    else:
      #WITH CONCEPTOR
      d_cn = weat_effect_size(white, black, pleasant, unpleasant, all_words)
      p_cn = weat_p_value(white, black, pleasant, unpleasant, all_words, 1000)
      print("With debiasing: ", embd, subspace)
      print('WEAT d = ', d_cn)
      print('WEAT p = ', p_cn)
    
    row = [embd, subspace, d_cn, p_cn]
    results.append(row)
    
pd.DataFrame(np.array(results), columns = ['Embedding', 'Subspace', 'Effect Size', 'p-value'])

## Bolukbasi hard debiasing

In [0]:
def doPCA(pairs, mat, index, num_components = 5):
    matrix = []
    for a, b in pairs:
        center = (mat[index[a.lower()]] + mat[index[b.lower()]])/2
        matrix.append(mat[index[a.lower()]] - center)
        matrix.append(mat[index[b.lower()]] - center)
    matrix = np.array(matrix)
    pca = PCA(n_components = num_components)
    pca.fit(matrix)
    # bar(range(num_components), pca.explained_variance_ratio_)
    return pca

def drop(u, v):
    return u - v * u.dot(v) / v.dot(v)
  
def normalize(all_words_mat):
    all_words_mat /= np.linalg.norm(all_words_mat, axis=1)[:, np.newaxis]
    return all_words_mat

In [0]:
def debias(all_words_mat, all_words_index, gender_specific_words, definitional, equalize):
    gender_direction = doPCA(definitional, all_words_mat, all_words_index).components_[0]
    specific_set = set(gender_specific_words)
    for w in list(all_words_index.keys()):
        if w not in specific_set:
            all_words_mat[all_words_index[w.lower()]] = drop(all_words_mat[all_words_index[w.lower()]], gender_direction)
    all_words_mat = normalize(all_words_mat)
    candidates = {x for e1, e2 in equalize for x in [(e1.lower(), e2.lower()),
                                                     (e1.title(), e2.title()),
                                                     (e1.upper(), e2.upper())]}
    print(candidates)
    for (a, b) in candidates:
        if (a.lower() in all_words_index and b.lower() in all_words_index):
            y = drop((all_words_mat[all_words_index[a.lower()]] + all_words_mat[all_words_index[b.lower()]]) / 2, gender_direction)
            z = np.sqrt(1 - np.linalg.norm(y)**2)
            if (all_words_mat[all_words_index[a.lower()]] - all_words_mat[all_words_index[b.lower()]]).dot(gender_direction) < 0:
                z = -z
            all_words_mat[all_words_index[a.lower()]] = z * gender_direction + y
            all_words_mat[all_words_index[b.lower()]] = -z * gender_direction + y
    all_words_mat = normalize(all_words_mat)
    return all_words_mat


In [0]:
import numpy as np
import pandas as pd

resourceFile = ''
wikiWordsPath = resourceFile + 'SIF/auxiliary_data/enwiki_vocab_min200.txt' # https://github.com/PrincetonML/SIF/blob/master/auxiliary_data/enwiki_vocab_min200.txt

%cd debiaswe_tutorial/debiaswe/
# Lets load some gender related word lists to help us with debiasing
with open('./data/definitional_pairs.json', "r") as f:
    defs = json.load(f) #gender definitional words
print("definitional", defs)
defs_list = []
for pair in defs:
  defs_list.append(pair[0])
  defs_list.append(pair[1])

with open('./data/equalize_pairs.json', "r") as f:
    equalize_pairs = json.load(f) 
print("Equalize pairs", equalize_pairs)

%cd ../../
!ls

all_embd = ['glove', 'word2vec', 'fasttext']
all_subspace = ['without_debiasing', 'gender_list_extended','gender_list_propernouns','gender_list_pronouns', 'gender_list_all']
#all_subspace = ['without_debiasing', 'race_list']

math = WEATLists.W_7_Math
arts = WEATLists.W_7_Arts
male = WEATLists.W_7_Male_terms
female = WEATLists.W_7_Female_terms

# white = WEATLists.W_5_Unused_full_list_European_American_names
# black = WEATLists.W_5_Unused_full_list_African_American_names
# pleasant = WEATLists.W_5_Pleasant
# unpleasant = WEATLists.W_5_Unpleasant

#print(career, family, male, female)
results = []

for embd in all_embd:
  #Initialize the embeddings to be used
  curr_embd = eval(embd)
  
  #Load all embeddings in a matrix of all words in the wordlist
  if embd == 'elmo' or embd == 'bert':
    all_words_mat, all_words_index, _ = pick_embeddings(brown_corpus, curr_embd)
    print("All mat: ", np.array(all_words_mat).shape)
    print("Number of words: ", len(list(all_words_index.keys())))
  else:
    all_words_index, all_words_mat = load_all_vectors(curr_embd, wikiWordsPath)
  
  for subspace in all_subspace:
    
    if subspace != 'without_debiasing' and subspace != 'gender_list_and':
      subspace_words_list = eval(subspace)

    
    if subspace != 'without_debiasing':
      #Load all embeddings of the subspace as a matrix
      if embd == 'elmo' or embd == 'bert':
        #subspace_words_mat = load_subspace_vectors_contextual(all_words_mat, all_words_index, subspace_words_list)
        all_words_cn = debias(all_words_mat, all_words_index, subspace_words_list, defs, equalize_pairs)
        print("Subspace mat: ", np.array(subspace_words_mat).shape)
      else:
        #subspace_words_mat = load_subspace_vectors(curr_embd, subspace_words_list)
        all_words_cn = debias(all_words_mat, all_words_index, subspace_words_list, defs, equalize_pairs)
    else:
      all_words_cn = all_words_mat
    all_words_cn = np.array(all_words_cn)
    print("All CN: ", all_words_cn.shape)
    #Store all conceptored words in a dictonary
    all_words = {}
    for word, index in all_words_index.items():
      #print(word, index)
      if embd == 'elmo' or embd == 'bert':
        all_words[word] = np.mean([all_words_cn[i,:] for i in index], axis = 0)
      else:
        all_words[word] = all_words_cn[index,:]
    
    if subspace == 'without_debiasing':
      #WITHOUT CONCEPTOR
      d_cn = weat_effect_size(math, arts, male, female, all_words)
      p_cn = weat_p_value(math, arts, male, female, all_words, 1000)
      print("Without debiasing")
      print('WEAT d = ', d_cn)
      print('WEAT p = ', p_cn)
    else:
      #WITH CONCEPTOR
      d_cn = weat_effect_size(math, arts, male, female, all_words)
      p_cn = weat_p_value(math, arts, male, female, all_words, 1000)
      print("With debiasing: ", embd, subspace)
      print('WEAT d = ', d_cn)
      print('WEAT p = ', p_cn)
    
    row = [embd, subspace, d_cn, p_cn]
    results.append(row)
    
pd.DataFrame(np.array(results), columns = ['Embedding', 'Subspace', 'Effect Size', 'p-value'])